<a href="https://colab.research.google.com/github/abellizzistudent/CST383_Project/blob/main/SAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyreadstat

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
#import pyreadstat
from matplotlib import rcParams

# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html
sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# San Francisco Bite Data 2014-2018

San Francisco Bite Data, 2014-2018 - Total rows: 3863
Exported PDF to Exel via Adobe Acrobat
Removed rows that contained only total by breed sections - 267 rows
Removed header rows that appeard within the doc as a PDF - 475 rows
Split bite severity into two seperate columns - BITE_CODE and BITE_SEVERITY
***Retaining both columns for now, both are not necessary since they contain the same infomation
Replaced alpha "U" code in BITE_SEVERITY for UNKNOWN to 9 to match numeric convention of hte column
Removed spaces in BITE_SEVERITY alpha codes for ease of search
Removed BITE column because each row value for that column was 1. Each row represents a single event already, column is not necessary.
Total Rows: 3626
Total Cols: 5
Change column names to all lowercase
Imported as CSV - sf-raw-data-dog-bites-2014-2018-CLEAN.csv


In [ ]:
df_sf = pd.read_csv("/content/sf-raw-data-dog-bites-2014-2018-CLEAN.csv")
df_sf.head()
###breed_group may be an unnecessary column:
#breed_group = df_sf.breed_group
#print(breed_group.unique().size)
#primary_breed = df_sf.primary_breed
#print(primary_breed.unique().size)




# New York Bite Data 2015-2021

Removed columns Borough, ZipCode, Species
Combined Gender and Spay/Neuter columns to lmatch format of DF_SF gender column
Removed Key column and Date of bite column
Ages modified to standard values, removed alphas and turned numbers into decimal value in years.
Added Bad_Data column to tag potentially bad rows that could be thrown out of the dataset.
Added breed_group column to match df_sf dataset and clarify primary_breed values that specify a group.
added Multi-Dog column for entries invloving more than one dog in description. Low numbers, may consider dropping these rows.

Cleanup is not complete, need to standardize breed column and breed_group column

The data does not include whether one dog is a multi-offender. 



In [ ]:
df_ny = pd.read_csv("/content/ny-dog-bites-2015-2021-CLEAN.csv")
df_ny.head()